In [20]:
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import Adam
import numpy as np
import pandas as pd
import random
from skimage.restoration import denoise_wavelet
from nexcsi import decoder

# Initial data

In [19]:
device = "raspberrypi"
raw_data_path = "./Data8_Train/room/Red"
processed_path = "./processed"
classes = ["EMPTY", "WALK", "STAND", "SIT"]
activity_code = {
    "EMPTY": "1",
    "WALK": "4",
    "STAND": "3",
    "SIT": "5",
}
activity_vector = {
    "EMPTY": [0, 0, 0, 1],
    "WALK": [0, 0, 1, 0],
    "STAND": [0, 1, 0, 0],
    "SIT": [1, 0, 0, 0],
}
records_num = 1800
samples_num = records_num * len(classes)
bandwidth = 40
carriers_num = 108
timestamps_num = 150
x = []
y = []

# Signal Filters

In [16]:
def median_absolute_deviation(x):
    """
    Returns the median absolute deviation from the window's median
    :param x: Values in the window
    :return: MAD
    """
    return np.median(np.abs(x - np.median(x)))

def hampel_d(ts, window_size=5, n=3, imputation=False):

    """
    Median absolute deviation (MAD) outlier in Time Series
    :param ts: a pandas Series object representing the timeseries
    :param window_size: total window size will be computed as 2*window_size + 1
    :param n: threshold, default is 3 (Pearson's rule)
    :param imputation: If set to False, then the algorithm will be used for outlier detection.
        If set to True, then the algorithm will also imput the outliers with the rolling median.
    :return: Returns the outlier indices if imputation=False and the corrected timeseries if imputation=True
    """

    if type(ts) != pd.Series:
        raise ValueError("Timeserie object must be of tyme pandas.Series.")

    if type(window_size) != int:
        raise ValueError("Window size must be of type integer.")
    else:
        if window_size <= 0:
            raise ValueError("Window size must be more than 0.")

    if type(n) != int:
        raise ValueError("Window size must be of type integer.")
    else:
        if n < 0:
            raise ValueError("Window size must be equal or more than 0.")

    # Copy the Series object. This will be the cleaned timeserie
    ts_cleaned = ts.copy()

    # Constant scale factor, which depends on the distribution
    # In this case, we assume normal distribution
    k = 1.4826

    rolling_ts = ts_cleaned.rolling(window_size*2, center=True)
    rolling_median = rolling_ts.median().fillna(method='bfill').fillna(method='ffill')
    rolling_sigma = k*(rolling_ts.apply(median_absolute_deviation).fillna(method='bfill').fillna(method='ffill'))

    outlier_indices = list(
        np.array(np.where(np.abs(ts_cleaned - rolling_median) >= (n * rolling_sigma))).flatten())

    if imputation:
        ts_cleaned[outlier_indices] = rolling_median[outlier_indices]
        return ts_cleaned

    return outlier_indices



def hampel_filter_light(data, window_size=3, n_sigma=3):
    n = len(data)
    filtered = np.zeros(n)
    for i in range(n):
        lower = max(0, i - window_size)
        upper = min(n, i + window_size)
        x = data[lower:upper]
        median = np.median(x)
        deviation = np.abs(x - median)
        MAD = np.median(deviation)
        threshold = n_sigma * MAD
        if np.abs(data[i] - median) > threshold:
            filtered[i] = median
        else:
            filtered[i] = data[i]
    return filtered

# Readers and processors

In [17]:
def read_pcap(label, idx):
  samples_r = decoder(device).read_pcap(raw_data_path + '/' + label + '/R_' + label + '_00' + activity_code[label] + '_P150_' + str(idx).zfill(4) + '.pcap', bandwidth=bandwidth)

  return decoder(device).unpack(samples_r['csi'], zero_nulls=False)

def read_pcap_from(path):
    samples_r = decoder(device).read_pcap(path, bandwidth=bandwidth)

    return decoder(device).unpack(samples_r['csi'], zero_nulls=False)

def read_csv(label, idx):
  path = processed_path + '/' + label + '/' + label + '_' + str(idx + 1).zfill(4) + '.csv'
  return np.genfromtxt(path, delimiter=',')

def proccess_csi(csi):
  csi = np.delete(csi, csi.dtype.metadata['nulls'] + csi.dtype.metadata['pilots'], axis=1)
  csi = np.abs(csi)

  for i in range(len(csi)):
    csi[i] = list(hampel_d(pd.Series(csi[i]), 3, imputation=True))

  csi = denoise_wavelet(csi, wavelet='sym6', mode='soft', wavelet_levels=3, method='BayesShrink', rescale_sigma='True')

  return csi

def proccess_csi_light(csi):
  csi = np.delete(csi, csi.dtype.metadata['nulls'] + csi.dtype.metadata['pilots'], axis=1)
  csi = np.abs(csi)

  for i in range(len(csi)):
    csi[i] = hampel_filter_light(csi[i])

  csi = denoise_wavelet(csi, wavelet='sym6', mode='soft', wavelet_levels=3, method='BayesShrink', rescale_sigma='True')

  return csi

def read_and_process(path, separator = 9000):
    csi = read_pcap_from(path)
    csi = proccess_csi(csi)
    csi = csi[:separator]

    return np.array(np.split(csi.copy(), int(separator / timestamps_num)))

# Read dataset Data from CSV (preprocessed)

In [18]:
def read_csv(label, idx):
  path = processed_path + '/' + label + '/' + label + '_' + str(idx + 1).zfill(4) + '.csv'
  return np.genfromtxt(path, delimiter=',')

for label in classes:
  for i in range(0, records_num):
    y.append(label)
    x.append(read_csv(label, i))

for i in range(len(y)):
  y[i] = activity_vector[y[i]]

FileNotFoundError: ./processed/EMPTY/EMPTY_0001.csv not found.

# Or Read dataset Data from raw .pcap (and preprocess it)

In [ ]:
for label in classes:
  for i in range(1, records_num + 1):
    y.append(label)
    x.append(read_pcap(label, i))

for i in range(len(y)):
  y[i] = activity_vector[y[i]]

for i in range(len(x)):
  x[i] = proccess_csi(x[i])

for i in range(len(classes)):
  for ri in range(0, records_num):
    label = classes[i]
    idx = records_num*i + ri
    path = processed_path + '/' + label + '/' + label + '_' + str(ri + 1).zfill(4) + '.csv'
    np.savetxt(path, np.asarray(x[idx]), delimiter=",")

# Generate train and test data

In [8]:
y1 = np.array(y.copy())
x1 = np.array(x.copy())

t_i = list(range(0, len(y)))
random.shuffle(t_i)

for i in range(len(y)):
  y1[i] = y[t_i[i]]
  x1[i] = x[t_i[i]]

train_start_idx = 0
train_end_idx = int(len(y)*0.8)
test_start_idx = train_end_idx + 1
test_end_idx = samples_num - 1

x_train = np.array(x1[train_start_idx:train_end_idx])
y_train = np.array(y1[train_start_idx:train_end_idx])
x_test = np.array(x1[test_start_idx:test_end_idx])
y_test = np.array(y1[test_start_idx:test_end_idx])

print(y_train)

[[1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 ...
 [0 1 0 0]
 [0 0 0 1]
 [0 1 0 0]]


In [9]:
opt = Adam(learning_rate=0.000001)

# Define model architecture
model = Sequential()
model.add(LSTM(128, input_shape=(timestamps_num, carriers_num), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Train model
#model.fit(x_train, y_train, batch_size=32, epochs=7, validation_data=(x_test, y_test))
# Train the model on your data
# model.fit(x_train_s, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

2023-04-12 22:52:21.894558: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-12 22:52:21.920391: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-12 22:52:21.920864: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-12 22:52:21.923301: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-12 22:52:21.923664: I tensorflow/compile

In [10]:
# Train the model with the specified number of epochs and batch size
model.fit(np.array(x_train), np.array(y_train), epochs=150, batch_size=128)

# Evaluate the model on the test set
score = model.evaluate(x_test, y_test, batch_size=32)

# Print the test accuracy
print("Test accuracy:", score[1])

Epoch 1/150


2023-04-12 22:53:02.804255: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-12 22:53:02.805143: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-12 22:53:02.806627: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

45/45 [==============================] - 4s 29ms/step - loss: 1.4079 - accuracy: 0.1906
Epoch 2/150
45/45 [==============================] - 1s 24ms/step - loss: 1.4040 - accuracy: 0.1960
Epoch 3/150
45/45 [==============================] - 1s 24ms/step - loss: 1.4005 - accuracy: 0.1981
Epoch 4/150
45/45 [==============================] - 1s 24ms/step - loss: 1.3963 - accuracy: 0.1990
Epoch 5/150
45/45 [==============================] - 1s 24ms/step - loss: 1.3908 - accuracy: 0.2054
Epoch 6/150
45/45 [==============================] - 1s 23ms/step - loss: 1.3887 - accuracy: 0.2097
Epoch 7/150
45/45 [==============================] - 1s 24ms/step - loss: 1.3846 - accuracy: 0.2165
Epoch 8/150
45/45 [==============================] - 1s 23ms/step - loss: 1.3798 - accuracy: 0.2172
Epoch 9/150
45/45 [==============================] - 1s 23ms/step - loss: 1.3773 - accuracy: 0.2196
Epoch 10/150
45/45 [==============================] - 1s 24ms/step - loss: 1.3727 - accuracy: 0.2297
Epoch 11/15

2023-04-12 22:56:05.233245: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-12 22:56:05.234473: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-12 22:56:05.236305: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

45/45 [==============================] - 1s 14ms/step - loss: 0.6355 - accuracy: 0.9590
Test accuracy: 0.9589707851409912


In [11]:
model.save('./models/cnn3.keras')

In [8]:
model = load_model('./models/cnn2.keras')

2023-04-14 17:03:06.540095: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-14 17:03:06.732126: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-14 17:03:06.732862: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-14 17:03:06.734970: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-14 17:03:06.735178: I tensorflow/compile

In [14]:
print(np.shape(np.array(x1)))

(7200, 150, 108)


# Processing and testing actual data from the room

In [9]:
packet_sit = read_and_process('./test/sit.pcap')
packet_walk = read_and_process('./test/walk.pcap')
packet_stand = read_and_process('./test/stand.pcap')
packet_empty = read_and_process('./test/empty.pcap')

In [10]:
model.predict(packet_sit)

2023-04-14 17:13:11.942117: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-14 17:13:11.943151: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-14 17:13:11.943878: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2/2 [==============================] - 3s 15ms/step


2023-04-14 17:13:14.935787: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


array([[0.41909856, 0.3271703 , 0.10040532, 0.15332578],
       [0.7183859 , 0.13372134, 0.07900436, 0.06888837],
       [0.33316675, 0.19361648, 0.36410722, 0.10910963],
       [0.3902576 , 0.26793328, 0.20018934, 0.14161982],
       [0.4041421 , 0.20982923, 0.2401485 , 0.14588022],
       [0.34229046, 0.21490432, 0.33415148, 0.10865381],
       [0.45176196, 0.26043636, 0.1985894 , 0.08921234],
       [0.33819565, 0.28137636, 0.27085462, 0.10957336],
       [0.35189256, 0.10842748, 0.4964842 , 0.04319577],
       [0.5608976 , 0.25489563, 0.10034455, 0.08386226],
       [0.6193366 , 0.19296804, 0.10045039, 0.08724497],
       [0.62240934, 0.19082971, 0.09652475, 0.09023616],
       [0.4041354 , 0.24582629, 0.24319997, 0.10683824],
       [0.38035223, 0.21807894, 0.28085354, 0.12071535],
       [0.32690743, 0.2690042 , 0.2482028 , 0.15588555],
       [0.40636566, 0.23145433, 0.25602573, 0.10615433],
       [0.50074667, 0.21891654, 0.16424225, 0.11609451],
       [0.2972662 , 0.33915862,

In [11]:
model.predict(packet_stand)

2/2 [==============================] - 0s 37ms/step


array([[0.32370362, 0.31142524, 0.26523185, 0.09963924],
       [0.3175016 , 0.29446137, 0.25502643, 0.13301055],
       [0.2638508 , 0.24664845, 0.37068588, 0.11881492],
       [0.37742108, 0.1896247 , 0.30812183, 0.12483234],
       [0.30501863, 0.27849606, 0.29363185, 0.12285345],
       [0.82997394, 0.06373736, 0.07002258, 0.0362661 ],
       [0.63440335, 0.18647622, 0.10468692, 0.0744335 ],
       [0.33967894, 0.34293634, 0.12022132, 0.19716343],
       [0.2816546 , 0.34880725, 0.2045532 , 0.16498497],
       [0.27559614, 0.20833649, 0.4200614 , 0.09600585],
       [0.7292721 , 0.12233413, 0.08331327, 0.06508051],
       [0.2961744 , 0.18331821, 0.42727536, 0.09323207],
       [0.6909223 , 0.14478666, 0.09460926, 0.06968174],
       [0.5651567 , 0.22372092, 0.10842707, 0.10269531],
       [0.6624264 , 0.17692184, 0.08915633, 0.07149537],
       [0.49591926, 0.26753893, 0.1151256 , 0.12141616],
       [0.3494325 , 0.17875418, 0.38622722, 0.08558612],
       [0.7835809 , 0.08143532,

In [12]:
model.predict(packet_walk)

2/2 [==============================] - 0s 37ms/step


array([[0.44975525, 0.23948915, 0.19126551, 0.11949013],
       [0.42482537, 0.2725803 , 0.21300717, 0.08958714],
       [0.39971069, 0.25883192, 0.21305169, 0.1284057 ],
       [0.41902572, 0.2558328 , 0.19152409, 0.13361734],
       [0.40761298, 0.23814614, 0.22038525, 0.13385561],
       [0.25139794, 0.32300496, 0.27716377, 0.14843333],
       [0.5207796 , 0.20493653, 0.12892827, 0.14535561],
       [0.3992982 , 0.25248057, 0.19862808, 0.1495931 ],
       [0.37962127, 0.24019578, 0.2119013 , 0.1682816 ],
       [0.41778916, 0.24318433, 0.22275896, 0.11626759],
       [0.5458268 , 0.15807937, 0.23095626, 0.06513759],
       [0.37899223, 0.295453  , 0.20871347, 0.11684135],
       [0.23426935, 0.383652  , 0.22625546, 0.1558232 ],
       [0.5264997 , 0.24781048, 0.09868243, 0.12700741],
       [0.3242163 , 0.31006873, 0.18600644, 0.1797085 ],
       [0.3975006 , 0.24839059, 0.20744969, 0.14665908],
       [0.4125955 , 0.26413757, 0.21373567, 0.10953132],
       [0.7022562 , 0.09706607,

In [13]:
model.predict(packet_empty)

2/2 [==============================] - 0s 43ms/step


array([[0.8359353 , 0.04230146, 0.0842602 , 0.03750303],
       [0.8542182 , 0.03524317, 0.08044513, 0.03009342],
       [0.7963921 , 0.06007487, 0.08312363, 0.06040948],
       [0.86207634, 0.03054426, 0.07924587, 0.0281335 ],
       [0.84945863, 0.03585526, 0.07594498, 0.03874115],
       [0.85211   , 0.03242475, 0.07781545, 0.03764979],
       [0.81222045, 0.04438248, 0.09309618, 0.05030084],
       [0.89113915, 0.02418866, 0.06074667, 0.02392556],
       [0.8816172 , 0.03015577, 0.06682513, 0.0214019 ],
       [0.7723219 , 0.07149473, 0.08528293, 0.07090045],
       [0.812346  , 0.04784256, 0.08659967, 0.05321181],
       [0.7622188 , 0.0716893 , 0.10476804, 0.06132391],
       [0.85014105, 0.03965206, 0.07207339, 0.03813352],
       [0.8913319 , 0.02552658, 0.0609456 , 0.02219588],
       [0.7998556 , 0.06554238, 0.08569609, 0.04890587],
       [0.8204845 , 0.0516321 , 0.08507801, 0.04280543],
       [0.8011846 , 0.06156729, 0.08864123, 0.04860693],
       [0.8761184 , 0.02470079,

In [41]:
from keras import backend as K
K.clear_session()